In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import numpy as np
import pandas as pd

import torch
from  torch import nn

In [3]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [4]:
y = pd.read_csv(labels_path, index_col='filename')

In [5]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [6]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [7]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [8]:
# data.show_batch(3, figsize=(6,6))

In [9]:
learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=MSELossFlat()
)#.to_fp16()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/ec2-user/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 78.2MB/s]


In [10]:
learn.path = Path('..')

In [1]:
learn.lr_find()
learn.recorder.plot()

In [16]:
learn.freeze_to(-1)
learn.fit_one_cycle(
    30, 
    slice(1e-2), 
    callbacks=[
        SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
        EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=3)
    ]
)

epoch,train_loss,valid_loss,time
0,1.042396,0.198158,01:11
1,0.604568,0.094170,01:10
2,0.315669,0.060653,01:11
3,0.202752,0.063023,01:11
4,0.169145,0.056395,01:11
5,0.142665,0.048790,01:11
6,0.105508,0.034817,01:11
7,0.084674,0.025406,01:11
8,0.064814,0.016685,01:11
9,0.073346,0.015950,01:11


Better model found at epoch 0 with valid_loss value: 0.19815798103809357.
Better model found at epoch 1 with valid_loss value: 0.09417025744915009.
Better model found at epoch 2 with valid_loss value: 0.060653455555438995.
Better model found at epoch 4 with valid_loss value: 0.05639549344778061.
Better model found at epoch 5 with valid_loss value: 0.048790331929922104.
Better model found at epoch 6 with valid_loss value: 0.03481731191277504.
Better model found at epoch 7 with valid_loss value: 0.025406161323189735.
Better model found at epoch 8 with valid_loss value: 0.016685357317328453.
Better model found at epoch 9 with valid_loss value: 0.015950292348861694.
Better model found at epoch 10 with valid_loss value: 0.012695029377937317.
Better model found at epoch 11 with valid_loss value: 0.012000830844044685.
Epoch 12: early stopping


In [11]:
learn = learn.load('fastai_model')
learn = learn.to_fp32()
learn.save('fastai_model')

In [12]:
learn.export('models/export.pkl')
learn = load_learner('../models', 'export.pkl')

In [23]:
# learn.show_results(rows=10)

### Sagemaker route

In [25]:
import tarfile
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker.predictor import RealTimePredictor, json_deserializer
from sagemaker.pytorch.model import PyTorchModel

with tarfile.open('../models/model.tar.gz', 'w:gz') as f:
    t = tarfile.TarInfo('models')
    t.type = tarfile.DIRTYPE
    f.addfile(t)
    f.add('../models/export.pkl', arcname='model.pkl')

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = f'sagemaker/{name_from_base("facial-features-model")}'
model_artefact = sagemaker_session.upload_data(
    path=str('../models/model.tar.gz'), 
    bucket=bucket, 
    key_prefix=prefix
)

In [ ]:
class ImagePredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(
            endpoint_name, 
            sagemaker_session=sagemaker_session, 
            serializer=None, 
            deserializer=json_deserializer, 
            content_type='image/jpeg'
        )

In [35]:
role = ###
# sagemaker.get_execution_role()

model = PyTorchModel(
    model_data=model_artefact, 
    name=name_from_base("facial-features-model"),
    role=role,
    framework_version='1.2.0', 
    entry_point='serve.py', 
    predictor_cls=ImagePredictor
)

In [36]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------------------------------------------------------------------------------------------------------------------------------!

In [11]:
url = "https://mtpcontent-dev.s3.amazonaws.com/ce99060e047d44279e0887270af5afe6.png"
img_bytes = requests.get(url).content
predictor.predict(img_bytes); response

### Bentoml Route

In [13]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(pip_dependencies=['fastai'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return str(result)

Overwriting pet_regression.py


In [14]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2019-12-03 00:10:20,372] INFO - BentoService bundle 'PetRegression:20191203000948_9971B8' created at: /tmp/bentoml-temp-nvz0lh45
[2019-12-03 00:10:20,851] INFO - BentoService bundle 'PetRegression:20191203000948_9971B8' created at: /home/ec2-user/bentoml/repository/PetRegression/20191203000948_9971B8


In [15]:
from bentoml import load

service = load(saved_path)

print(service.predict(data.get(0)))

[2019-12-03 00:10:21,249] WARNING - Module `pet_regression` already loaded, using existing imported module.
(ImagePoints (320, 320), tensor([[-0.5133, -0.5090],
        [-0.7960, -0.4386],
        [-0.8005, -0.2425],
        [-0.8656, -0.0839],
        [-0.5567, -0.0082],
        [-0.2026, -0.2832],
        [-0.5261, -0.3297],
        [-0.5509, -0.1878],
        [-0.2745, -0.3370],
        [-0.3003, -0.2733],
        [-0.3027, -0.2223]]), tensor([-0.5133, -0.5090, -0.7960, -0.4386, -0.8005, -0.2425, -0.8656, -0.0839,
        -0.5567, -0.0082, -0.2026, -0.2832, -0.5261, -0.3297, -0.5509, -0.1878,
        -0.2745, -0.3370, -0.3003, -0.2733, -0.3027, -0.2223]))


In [16]:
!pip install --upgrade {saved_path}

Processing /home/ec2-user/bentoml/repository/PetRegression/20191203000948_9971B8
  Created wheel for PetRegression: filename=PetRegression-20191203000948_9971B8-cp36-none-any.whl size=51631513 sha256=c271ae9c33f2839caa1abe52f8f749a85efd02aa05e1060635856192072bc5b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-argdva34/wheels/88/d1/46/c6f7232025425f80ca5bdfb8aeb55e81531d8bef1bdc801a12
Successfully built PetRegression


In [17]:
!PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

(ImagePoints (320, 320), tensor([[-0.4128,  0.1322],
        [-0.7365,  0.1464],
        [-0.6189,  0.2665],
        [-0.7248,  0.4663],
        [-0.4005,  0.3247],
        [-0.2695,  0.2497],
        [-0.3914,  0.2126],
        [-0.4023,  0.3228],
        [-0.2775,  0.2574],
        [-0.2962,  0.2582],
        [-0.2722,  0.2691]]), tensor([-0.4128,  0.1322, -0.7365,  0.1464, -0.6189,  0.2665, -0.7248,  0.4663,
        -0.4005,  0.3247, -0.2695,  0.2497, -0.3914,  0.2126, -0.4023,  0.3228,
        -0.2775,  0.2574, -0.2962,  0.2582, -0.2722,  0.2691]))


In [18]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20191203000948_9971B8


In [29]:
!bentoml --verbose deployments delete facial-features-detector --force

[2019-12-03 00:33:26,163] DEBUG - Using BentoML with local Yatai server
[2019-12-03 00:33:26,435] DEBUG - Upgrading tables to the latest revision
[2019-12-03 00:33:26,809] DEBUG - Deleting AWS Lambda deployment
[2019-12-03 00:33:26,853] DEBUG - Deleting AWS CloudFormation: default-facial-features-detector that includes Lambda function and related resources
Successfully deleted deployment "facial-features-detector"


In [ ]:
!bentoml --verbose deployment create facial-features-detector --bento {bento_tag} --platform aws-lambda 

[2019-12-03 00:15:06,284] DEBUG - Using BentoML with local Yatai server
[2019-12-03 00:15:06,407] DEBUG - Upgrading tables to the latest revision
[2019-12-03 00:15:07,161] DEBUG - Creating s3 bucket: btml-default-facial-features-detector-d02d88 in region us-west-2
[2019-12-03 00:15:08,174] DEBUG - Uploading artifacts to S3 bucket
[2019-12-03 00:15:08,175] DEBUG - This lambda deployment requires uploading artifacts to s3
[2019-12-03 00:15:08,179] DEBUG - Uploading __init__.py to s3 btml-default-facial-features-detector-d02d88/default/facial-features-detector/artifacts/PetRegression/20191203000948_9971B8/__init__.py
[2019-12-03 00:15:08,519] DEBUG - Uploading pet_regressor.pkl to s3 btml-default-facial-features-detector-d02d88/default/facial-features-detector/artifacts/PetRegression/20191203000948_9971B8/pet_regressor.pkl
[2019-12-03 00:15:15,826] DEBUG - Created temporary directory: /tmp/bentoml-temp-tyqb7zys
[2019-12-03 00:15:15,827] DEBUG - Generating cloudformation template.yaml for 

In [28]:
print("jdlf")

jdlf
